# OpenAI Tool Calling Agent - Dummy Weather Example

* Collaborator: Roberto Rodriguez @Cyb3rWard0g

## Import Environment Variables

In [1]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

## Enable Logging

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

## Define Tools

In [3]:
from dapr_agents import tool
from pydantic import BaseModel, Field

class GetWeatherSchema(BaseModel):
    location: str = Field(description="location to get weather for")

@tool(args_model=GetWeatherSchema)
def get_weather(location: str) -> str:
    """Get weather information for a specific location."""
    import random
    temperature = random.randint(60, 80)
    return f"{location}: {temperature}F."

class JumpSchema(BaseModel):
    distance: str = Field(description="Distance for agent to jump")

@tool(args_model=JumpSchema)
def jump(distance: str) -> str:
    """Jump a specific distance."""
    return f"I jumped the following distance {distance}"

tools = [get_weather,jump]

## Initialize Agent

In [4]:
from dapr_agents import ReActAgent

AIAgent = ReActAgent(
    name="Rob",
    role= "Weather Assistant",
    tools=tools
)

INFO:dapr_agents.llm.openai.client.base:Initializing OpenAI client...
INFO:dapr_agents.tool.executor:Tool registered: GetWeather
INFO:dapr_agents.tool.executor:Tool registered: Jump
INFO:dapr_agents.tool.executor:Tool Executor initialized with 2 registered tools.
INFO:dapr_agents.agent.base:Constructing system_prompt from agent attributes.
INFO:dapr_agents.agent.base:Using system_prompt to create the prompt template.
INFO:dapr_agents.agent.base:Pre-filled prompt template with attributes: ['name', 'role', 'goal']


In [5]:
AIAgent.prompt_template

ChatPromptTemplate(input_variables=['chat_history'], pre_filled_variables={'name': 'Rob', 'role': 'Weather Assistant', 'goal': 'Help humans'}, messages=[('system', '# Today\'s date is: March 04, 2025\n\n## Name\nYour name is {{name}}.\n\n## Role\nYour role is {{role}}.\n\n## Goal\n{{goal}}.\n\n## Tools\nYou have access ONLY to the following tools:\nGetWeather: Get weather information for a specific location.. Args schema: {\'location\': {\'description\': \'location to get weather for\', \'type\': \'string\'}}\nJump: Jump a specific distance.. Args schema: {\'distance\': {\'description\': \'Distance for agent to jump\', \'type\': \'string\'}}\n\nIf you think about using tool, it must use the correct tool JSON blob format as shown below:\n```\n{\n    "name": $TOOL_NAME,\n    "arguments": $INPUT\n}\n```\n\n## ReAct Format\nThought: Reflect on the current state of the conversation or task. If additional information is needed, determine if using a tool is necessary. When a tool is required,

In [6]:
AIAgent.tools[0].name

'GetWeather'

## Run Agent

In [7]:
AIAgent.run("Hi my name is Roberto")

INFO:dapr_agents.agent.base:Pre-filled prompt template with variables: dict_keys(['chat_history'])
INFO:dapr_agents.agent.patterns.react.base:Iteration 1/10 started.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
Hi my name is Roberto

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agent.patterns.react.base:No action specified; continuing with further reasoning.
INFO:dapr_agents.agent.patterns.react.base:Iteration 2/10 started.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


Thought: Hello Roberto! How can I assist you today with the weather?


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agent.patterns.react.base:Agent provided a direct final answer.


Thought: Answer: Hello Roberto! How can I assist you today with the weather?

--------------------------------------------------------------------------------

assistant:
Hello Roberto! How can I assist you today with the weather?


'Hello Roberto! How can I assist you today with the weather?'

In [8]:
AIAgent.chat_history

[{'role': 'user', 'content': 'Hi my name is Roberto'},
 {'content': 'Hello Roberto! How can I assist you today with the weather?',
  'role': 'assistant'}]

In [9]:
AIAgent.run("What is the weather in Virgina, New York and Washington DC?")

INFO:dapr_agents.agent.base:Pre-filled prompt template with variables: dict_keys(['chat_history'])
INFO:dapr_agents.agent.patterns.react.base:Iteration 1/10 started.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
What is the weather in Virgina, New York and Washington DC?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agent.patterns.react.base:Executing GetWeather with arguments {'location': 'Virginia'}
INFO:dapr_agents.tool.executor:Attempting to execute tool: GetWeather
INFO:dapr_agents.tool.executor:Tool 'GetWeather' executed successfully.
INFO:dapr_agents.agent.patterns.react.base:Thought:I will need to gather the current weather information for both Virginia and Washington, D.C. by using the GetWeather tool.
Action:{'name': 'GetWeather', 'arguments': {'location': 'Virginia'}}
Observation:Virginia: 74F.
INFO:dapr_agents.agent.patterns.react.base:Iteration 2/10 started.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


Thought: I will need to gather the current weather information for both Virginia and Washington, D.C. by using the GetWeather tool.
Action: {"name": "GetWeather", "arguments": {"location": "Virginia"}}
Observation: Virginia: 74F.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agent.patterns.react.base:Executing GetWeather with arguments {'location': 'New York'}
INFO:dapr_agents.tool.executor:Attempting to execute tool: GetWeather
INFO:dapr_agents.tool.executor:Tool 'GetWeather' executed successfully.
INFO:dapr_agents.agent.patterns.react.base:Thought:
Action:{'name': 'GetWeather', 'arguments': {'location': 'New York'}}
Observation:New York: 65F.
INFO:dapr_agents.agent.patterns.react.base:Iteration 3/10 started.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agent.patterns.react.base:Executing GetWeather with arguments {'location': 'Washington DC'}
INFO:dapr_agents.tool.executor:Attempting to execute tool: GetWeather
INFO:dapr_agents.tool.executor:Tool 'GetWeather' executed successfully.
INFO:dapr_agents.agent.patterns.react.base:Thought:
Action:{'name': 'GetWeather', 'arguments': {'location': 'Washington DC'}}
Observation:Washington DC: 66F.
INFO:dapr_agents.agent.patterns.react.base:Iteration 4/10 started.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


Thought: 
Action: {"name": "GetWeather", "arguments": {"location": "Washington DC"}}
Observation: Washington DC: 66F.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agent.patterns.react.base:Agent provided a direct final answer.


Thought: I now have sufficient information to answer the question. 

Answer: The current weather is as follows:
- Virginia: 74°F
- New York: 65°F
- Washington, D.C.: 66°F

--------------------------------------------------------------------------------

assistant:
The current weather is as follows:
- Virginia: 74°F
- New York: 65°F
- Washington, D.C.: 66°F


'The current weather is as follows:\n- Virginia: 74°F\n- New York: 65°F\n- Washington, D.C.: 66°F'

In [10]:
AIAgent.run("What places did you already help me with the weather?")

INFO:dapr_agents.agent.base:Pre-filled prompt template with variables: dict_keys(['chat_history'])
INFO:dapr_agents.agent.patterns.react.base:Iteration 1/10 started.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


user:
What places did you already help me with the weather?

--------------------------------------------------------------------------------



INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agent.patterns.react.base:No action specified; continuing with further reasoning.
INFO:dapr_agents.agent.patterns.react.base:Iteration 2/10 started.
INFO:dapr_agents.llm.openai.chat:Invoking ChatCompletion API.


Thought: You asked about the weather in Virginia, New York, and Washington, D.C., and I provided you with the current temperatures for those locations.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:dapr_agents.llm.openai.chat:Chat completion retrieved successfully.
INFO:dapr_agents.agent.patterns.react.base:Agent provided a direct final answer.


Thought: Answer: I helped you with the weather for Virginia, New York, and Washington, D.C.

--------------------------------------------------------------------------------

assistant:
I helped you with the weather for Virginia, New York, and Washington, D.C.


'I helped you with the weather for Virginia, New York, and Washington, D.C.'

In [11]:
AIAgent.chat_history

[{'role': 'user', 'content': 'Hi my name is Roberto'},
 {'content': 'Hello Roberto! How can I assist you today with the weather?',
  'role': 'assistant'},
 {'role': 'user',
  'content': 'What is the weather in Virgina, New York and Washington DC?'},
 {'content': 'The current weather is as follows:\n- Virginia: 74°F\n- New York: 65°F\n- Washington, D.C.: 66°F',
  'role': 'assistant'},
 {'role': 'user',
  'content': 'What places did you already help me with the weather?'},
 {'content': 'I helped you with the weather for Virginia, New York, and Washington, D.C.',
  'role': 'assistant'}]